In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import FlowCytometryTools as fct

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('../')
from pipeline_process import common
from pipeline_process.facs import constants, FACSProcessor, FACSUnmixer

FITC, SSC, FSC = constants.FITC, constants.SSC, constants.FSC

In [ ]:
FACS_DATA_ROOT = '/Users/keith.cheveralls/Box-cache/FACS_data/'

control_dir = os.path.join(FACS_DATA_ROOT, 'WT_plate19mNG')
sample_dir = os.path.join(FACS_DATA_ROOT, 'Plate19_FCS', 'profile_data')

In [ ]:
def plt_hist(bins, counts):
    binwidth = bins[1] - bins[0]
    x = bins[1:] - binwidth/2
    plt.plot(x, counts)

In [ ]:
# platemap (well_id and target_name)
platemap_filepath = '/Users/keith.cheveralls/Box-cache/LibraryDatabase/mNGplate19_Ref.csv'
platemap = common.file_utils.read_and_validate_platemap(platemap_filepath)

In [ ]:
# initialize the processor
p = FACSProcessor(sample_dir, control_dir)

In [ ]:
# load all negative controls
controls = p.load_controls()

In [ ]:
# concatenate all negative control values
all_values, all_means = p.concatenate_controls()

In [ ]:
# reference distribution
counts, edges = np.histogram(all_values, bins=np.arange(-2000, 2000, 100), density=True)
plt_hist(edges, counts)

xref = edges[1:]  - (edges[1] - edges[0])/2
yref = counts

In [ ]:
# sample distribution
d = p.load_sample('A3')
counts, edges = np.histogram(d.data[FITC], bins=np.arange(0, 10000, 150), density=True)

xs = edges[1:] - (edges[1] - edges[0])/2
ys = counts

plt.plot(xs, ys)

In [ ]:
m = FACSUnmixer(xref, yref, xs, ys, fit_window=(0, 2800))

In [ ]:
result = m.fit()
result.x

In [ ]:
plt.plot(xs, ys)
plt.plot(xs, m.predict(*result.x))
plt.plot(xs, ys - m.predict(*result.x))
# plt.gca().set_ylim([0, .0001])